In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from cvxopt import matrix, solvers
import cv2

from sklearn import svm

In [4]:
def get_img_data(dir_path):
    feature_len = 768
    desired_width = 16
    desired_height = 16
    data = np.empty((feature_len, 0))

    files = os.listdir(dir_path)

    for file in files:
        image = cv2.imread(os.path.join(dir_path, file))
        image = cv2.resize(image, (desired_width, desired_height))
        np_array = np.array(image)
        np_vector = np_array.flatten().reshape((feature_len, 1))
        data = np.concatenate((data, np_vector), axis=1)
    
    data = data/255
    
    return data

def format_data(dir_path_1, dir_path_2):
    class_1_data = get_img_data(dir_path_1)         # +1
    class_2_data = get_img_data(dir_path_2)         # -1

    pos_labels = np.full((1, class_1_data.shape[1]), 1.0)
    neg_labels = np.full((1, class_2_data.shape[1]), -1.0)
    
    data_x = np.concatenate((class_1_data, class_2_data), axis=1)
    data_y = np.concatenate((pos_labels, neg_labels), axis=1)

    return data_x, data_y

training_data_x, training_data_y = format_data('train/1/', 'train/2/')


In [5]:
def get_matrices_linear(data_x, data_y, c):
    
    m = data_x.shape[1]

    matrix_yx = data_y * data_x
    matrix_p = np.matmul(matrix_yx.T, matrix_yx)
    
    matrix_q = np.full((m,1), -1.0)

    matrix_b = np.array([[0.0]])

    matrix_a = data_y

    matrix_h_C = np.full((m, 1), c)
    matrix_h_0 = np.full((m, 1), 0.0)
    matrix_h = np.concatenate((matrix_h_C, matrix_h_0), axis=0)

    matrix_g_pos = np.eye(m, dtype=float)
    matrix_g_neg = (-1) * np.eye(m, dtype=float)
    matrix_g = np.concatenate((matrix_g_pos, matrix_g_neg), axis=0)

    return matrix(matrix_p), matrix(matrix_q), matrix(matrix_g), matrix(matrix_h), matrix(matrix_a), matrix(matrix_b)

def optimise(p, q, g, h, a, b):
    solution = solvers.qp(p, q, g, h, a, b)
    return solution

In [6]:
p, q, g, h, a, b = get_matrices_linear(training_data_x, training_data_y, 1.0)
solution = optimise(p, q, g, h, a, b)
alpha = np.array(solution['x'])
alpha

     pcost       dcost       gap    pres   dres
 0: -1.0489e+03 -1.2518e+04  8e+04  3e+00  3e-11
 1: -6.8322e+02 -8.0216e+03  2e+04  5e-01  3e-11
 2: -4.8967e+02 -3.1466e+03  4e+03  1e-01  2e-11
 3: -4.0304e+02 -1.5654e+03  2e+03  5e-02  2e-11
 4: -3.7361e+02 -7.7703e+02  6e+02  1e-02  1e-11
 5: -3.8001e+02 -5.4972e+02  2e+02  4e-03  1e-11
 6: -3.9526e+02 -4.6864e+02  8e+01  1e-03  2e-11
 7: -4.0571e+02 -4.3399e+02  3e+01  2e-04  2e-11
 8: -4.1139e+02 -4.2094e+02  1e+01  1e-05  2e-11
 9: -4.1426e+02 -4.1682e+02  3e+00  3e-06  2e-11
10: -4.1514e+02 -4.1559e+02  5e-01  4e-14  2e-11
11: -4.1533e+02 -4.1537e+02  3e-02  4e-13  2e-11
12: -4.1535e+02 -4.1535e+02  7e-04  2e-13  2e-11
13: -4.1535e+02 -4.1535e+02  1e-05  7e-14  2e-11
Optimal solution found.


array([[2.01878377e-10],
       [2.13313590e-10],
       [6.20430849e-10],
       ...,
       [1.97182100e-10],
       [4.21109982e-10],
       [3.20454949e-10]])

In [15]:
solution['primal objective']

-885.7576898279883

In [13]:
def get_matrices_gaussian(data_x, data_y, gamma, c):
    m = data_x.shape[1]
    temp = data_x.T
    # matrix p
    matrix_p = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            norm_sq = (np.linalg.norm(temp[i] - temp[j]))**2
            kernel = np.exp(-gamma*norm_sq)
            matrix_p[i, j] = data_y[0, i]*data_y[0, j]*kernel
    
    matrix_q = np.full((m,1), -1.0)

    matrix_b = np.array([[0.0]])

    matrix_a = data_y

    matrix_h_C = np.full((m, 1), c)
    matrix_h_0 = np.full((m, 1), 0.0)
    matrix_h = np.concatenate((matrix_h_C, matrix_h_0), axis=0)

    matrix_g_pos = np.eye(m, dtype=float)
    matrix_g_neg = (-1) * np.eye(m, dtype=float)
    matrix_g = np.concatenate((matrix_g_pos, matrix_g_neg), axis=0)

    return matrix(matrix_p), matrix(matrix_q), matrix(matrix_g), matrix(matrix_h), matrix(matrix_a), matrix(matrix_b)


In [14]:
p, q, g, h, a, b = get_matrices_gaussian(training_data_x, training_data_y, 0.001, 1.0)
solution = optimise(p, q, g, h, a, b)
alpha = np.array(solution['x'])
alpha

     pcost       dcost       gap    pres   dres
 0: -1.1860e+03 -1.2578e+04  7e+04  3e+00  2e-13
 1: -7.3522e+02 -8.1296e+03  1e+04  4e-01  2e-13
 2: -5.6455e+02 -1.6716e+03  1e+03  8e-03  2e-13
 3: -6.6510e+02 -1.3790e+03  7e+02  5e-03  2e-13
 4: -7.4937e+02 -1.1580e+03  4e+02  2e-03  2e-13
 5: -7.7440e+02 -1.1039e+03  3e+02  2e-03  2e-13
 6: -8.0472e+02 -1.0385e+03  2e+02  9e-04  2e-13
 7: -8.3295e+02 -9.7750e+02  1e+02  3e-04  2e-13
 8: -8.4698e+02 -9.5141e+02  1e+02  2e-04  2e-13
 9: -8.6050e+02 -9.2677e+02  7e+01  1e-04  2e-13
10: -8.7205e+02 -9.0692e+02  4e+01  5e-05  2e-13
11: -8.7866e+02 -8.9617e+02  2e+01  2e-05  2e-13
12: -8.8279e+02 -8.8974e+02  7e+00  6e-06  2e-13
13: -8.8527e+02 -8.8638e+02  1e+00  8e-07  2e-13
14: -8.8573e+02 -8.8578e+02  5e-02  2e-08  2e-13
15: -8.8576e+02 -8.8576e+02  1e-03  6e-10  2e-13
16: -8.8576e+02 -8.8576e+02  3e-05  1e-11  2e-13
Optimal solution found.


array([[1.35894135e-09],
       [3.77550910e-09],
       [9.99999981e-01],
       ...,
       [1.00276076e-09],
       [2.00270391e-09],
       [9.99999988e-01]])

In [ ]:
def 